### Gradient Boosting Regressor Implementation

* In this notebook, I implemented the gradient boosted trees to solve for a regression problem
* Then I verified the algorithm by comparing the results between Scikit-learn and my implementation. Note that values of the paramters (e.g. max_depth, n_estimators) used in both implementations are the same
* I referred to the pesudo code from this paper: https://statweb.stanford.edu/~jhf/ftp/trebst.pdf (Page 5-7) as well as Wikipedia: https://en.wikipedia.org/wiki/Gradient_boosting

#### 1. GradientBoostingRegressor from Scikit-learn, using the friedman data: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html

In [121]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, min_samples_leaf=1, \
                                max_depth=3, random_state=0, loss='ls').fit(X_train, y_train)
print "MSE using LS loss function: " + str(mean_squared_error(y_train, est.predict(X_train)))

MSE using LS loss function: 0.235100371411


#### 2. My GBRegressor implementation

In [122]:
def least_squares_pseudo_response(y_train, y_pred):
    return y_train - y_pred

def loss_function(loss):
    loss_dict = {'ls': least_squares_pseudo_response} # can add alternative loss function in
                                                      # the future
    return loss_dict[loss]

In [123]:
class GBRegressor():
    def __init__(self, n_estimators=10, min_sample=1, max_depth=3, \
                 learning_rate=0.1, loss_metric='ls'):
        '''
        Initialize the Gradient Boosting Regressor
        @param n_estimators: number of estimators
        @param min_sample: The minimum number of samples required in each leaf
        @param max_depth: maximum depth of the individual regression estimators
        @param learning_rate: learning rate shrinks the contribution of each tree
        @param loss_metric: loss function to be optimized
        '''
        self.n_estimators = n_estimators
        self.min_sample = min_sample
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.loss_metric = loss_metric
        
    def fit(self, x_train, y_train):
        '''
        Fit gradient boosting regressor
        '''
        y_pred = np.zeros(len(y_train))
        pseudo_response = loss_function(self.loss_metric)(y_train, y_pred) 
        tree_dict = dict()
        
        for i in xrange(self.n_estimators):           
            algo = DecisionTreeRegressor(min_samples_leaf=self.min_sample, max_depth=self.max_depth, \
                                         criterion='mse', random_state=0)
            algo.fit(x_train, pseudo_response)
            tree_dict[i] = algo
            resid_pred = algo.predict(x_train)
            y_pred += self.learning_rate * resid_pred
            pseudo_response = loss_function(self.loss_metric)(y_train, y_pred)        
        self.tree_dict = tree_dict
    
    def predict(self, x_test):
        '''
        Make predictions
        '''
        y_pred_test = np.zeros(len(x_test))
        for i in xrange(self.n_estimators):
            tree_stump_pred = self.tree_dict[i].predict(x_test)
            y_pred_test += self.learning_rate * tree_stump_pred
        return y_pred_test

In [124]:
algo = GBRegressor(n_estimators=100, learning_rate=0.1, min_sample=1, \
                   max_depth=3, loss_metric='ls')
algo.fit(X_train, y_train)
print "MSE using LS loss function: " + str(mean_squared_error(y_train, algo.predict(X_train)))

MSE using LS loss function: 0.235100511898


#### 3. Conclusion
* The MSE from GradientBoostingRegressor is 0.235100371411
* The MSE from GBRegressor is 0.235100511898. The results are almost the same
* Technically, each tree in gradient boosted trees should have a different weight, but it turned out using a fixed weight for each tree (in my implementation, the weight is 1) works fine.
* Future improvement can include: 
   1. User can use alternative loss functions e.g. Least-abosulote-deviation loss function and Huber loss function
   2. The implementation of GradientBoostingClassifier